In [5]:
import h5py
import torch
app = h5py.File('data/Youtube2Text/feats/image_resnet101_imagenet_fps_max60.hdf5','r')
mot = h5py.File('data/Youtube2Text/feats/motion_resnext101_kinetics_duration16_overlap8.hdf5','r')
min = 999
for k in app.keys():
    if torch.tensor(app[k]).size(0) < min : min = torch.tensor(app[k]).size(0)
print(min)
min = 999
for k in mot.keys():
    if torch.tensor(app[k]).size(0) < min : min = torch.tensor(app[k]).size(0)
print(min)

10
10


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class SP(nn.Module):
    """SP layer implementation
    
    Args:
        num_clusters : int
            The number of pseudo regions
        dim : int
            Dimension of pseudo regions
        alpha : float
            Parameter of initialization. Larger value is harder assignment.
        normalize_input : bool
            If true, pseudo regions-wise L2 normalization is applied to input.
    """
    def __init__(self, num_regions=64, dim=128, alpha=100.0, normalize_input=True):
        super().__init__()
        self.num_regions = num_regions
        self.dim = dim
        self.alpha = alpha
        self.normalize_input = normalize_input
        self.conv = nn.Conv2d(dim, num_regions, kernel_size=(1, 1), bias=True)
        self.centroids = nn.Parameter(torch.rand(num_regions, dim))
        self.init_weights()
    def init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    def forward(self, grids):
                   
        N, frame, C = grids.shape

        grids = grids.view(N, frame, 1, -1).permute(0,3,1,2).contiguous() # N dim grid 1
        print(grids.size)
        if self.normalize_input:
            grids = F.normalize(grids, p=2, dim=1)  # across descriptor dim

        soft_assign = self.conv(grids).view(N, self.num_regions, -1)
        soft_assign = F.softmax(soft_assign, dim=1)

        x_flatten = grids.view(N, C, -1)
        
        residual = x_flatten.expand(self.num_regions, -1, -1, -1).permute(1, 0, 2, 3).contiguous() - \
            self.centroids.expand(x_flatten.size(-1), -1, -1).permute(1, 2, 0).contiguous().unsqueeze(0)

        residual *= soft_assign.unsqueeze(2)
        p = residual.sum(dim=-1)

        p = F.normalize(p, p=2, dim=2)  # intra-normalization
        p = p.view(grids.size(0), -1)
        p = F.normalize(p, p=2, dim=1)  # L2 normalize

        return p.reshape(N, self.num_regions, -1)

f = SP(num_regions=8,dim=512)
a = torch.rand(10,20,512)
p = f(a)
print(p.size())

<built-in method size of Tensor object at 0x7f39571cfb90>
torch.Size([10, 8, 512])
